# This notebook shows initial exploration of the transaction data from housestockwatcher.com and senatestockwatcher.com

In [1]:
import pandas as pd

In [2]:
# Read in data. Initial data exploration revealed '--' was used sometimes for null values.
# We also know that there is date column 'transaction_date'.
senators = pd.read_csv('../data/senate_stocks.csv', index_col = 'senator', na_values = '--', 
                       parse_dates = ['transaction_date'])
reps = pd.read_csv('../data/house_stocks.csv', index_col = 'representative', na_values = '--',
                  parse_dates = ['transaction_date'])

In [3]:
list(senators.columns)

['transaction_date',
 'owner',
 'ticker',
 'asset_description',
 'asset_type',
 'type',
 'amount',
 'comment',
 'ptr_link']

In [4]:
list(reps.columns)

['disclosure_year',
 'disclosure_date',
 'transaction_date',
 'owner',
 'ticker',
 'asset_description',
 'type',
 'amount',
 'district',
 'ptr_link',
 'cap_gains_over_200_usd']

#### The shared columns in both files are transaction_date, owner, ticker, asset_description, amount, ptr_link. Need to examine asset_type and type

In [5]:
# Check the date conversion (just look at senators)
senators.dtypes

transaction_date     datetime64[ns]
owner                        object
ticker                       object
asset_description            object
asset_type                   object
type                         object
amount                       object
comment                      object
ptr_link                     object
dtype: object

In [6]:
# Look at the senators dataframe, especially transaction_date, type, asset_type
senators.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,ptr_link
senator,,,,,,,,,
Thomas R Carper,2021-05-26,Spouse,NaN,"HSBC Contingent Autocall BMY <div class=""text-...",Corporate Bond,Sale (Full),"$15,001 - $50,000",NaN,https://efdsearch.senate.gov/search/view/ptr/2...
Thomas R Carper,2021-05-26,Spouse,NaN,"HSBC Contingent Autocall BMY <div class=""text-...",Corporate Bond,Sale (Full),"$15,001 - $50,000",NaN,https://efdsearch.senate.gov/search/view/ptr/2...
Thomas R Carper,2021-05-07,Spouse,NaN,"CS Contingent Autocall CSCO <div class=""text-m...",Corporate Bond,Purchase,"$15,001 - $50,000",NaN,https://efdsearch.senate.gov/search/view/ptr/2...
Mike Rounds,2021-05-01,Joint,NaN,"Bird Dog Hospitality Fund 1, LLC <div class=""t...",Non-Public Stock,Purchase,"$15,001 - $50,000",NaN,https://efdsearch.senate.gov/search/view/ptr/3...
Shelley M Capito,2021-04-26,Spouse,WFC,Wells Fargo &amp; Company,Stock,Sale (Partial),"$1,001 - $15,000",NaN,https://efdsearch.senate.gov/search/view/ptr/8...


In [7]:
# Look at the reps dataframe, especially transaction_date, type
reps.head()

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,district,ptr_link,cap_gains_over_200_usd
representative,,,,,,,,,,,
Hon. Susie Lee,2020,04/28/2020,2020-02-24,joint,BHC,Bausch Health Companies Inc.,purchase,"$1,001 - $15,000",NV03,https://disclosures-clerk.house.gov/public_dis...,False
Hon. Susie Lee,2020,04/28/2020,2020-03-06,joint,BAH,Booz Allen Hamilton Holding Corporation,sale_full,"$1,001 - $15,000",NV03,https://disclosures-clerk.house.gov/public_dis...,True
Hon. Susie Lee,2020,04/28/2020,2020-03-23,joint,BYD,Boyd Gaming Corporation,sale_full,"$1,001 - $15,000",NV03,https://disclosures-clerk.house.gov/public_dis...,False
Hon. Susie Lee,2020,04/28/2020,2020-02-04,joint,CTLT,"Catalent, Inc.",purchase,"$1,001 - $15,000",NV03,https://disclosures-clerk.house.gov/public_dis...,False
Hon. Susie Lee,2020,04/28/2020,2020-03-18,joint,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",NV03,https://disclosures-clerk.house.gov/public_dis...,False


**Conclusions:** transaction date conversion worked. Asset type is not in representative data. Type needs to be standardized, different in both dataframes, but same information.

## We will want to focus on stocks. What is the relationship between Asset Type and Ticker?

In [8]:
asset_ticker = senators[['asset_type', 'ticker']]
asset_ticker.head(20)

,asset_type,ticker
senator,,
Thomas R Carper,Corporate Bond,NaN
Thomas R Carper,Corporate Bond,NaN
Thomas R Carper,Corporate Bond,NaN
Mike Rounds,Non-Public Stock,NaN
Shelley M Capito,Stock,WFC
"Jerry Moran,",Stock,VZ
Richard M Burr,PDF Disclosed Filing,NaN
Susan M Collins,Municipal Security,NaN
Susan M Collins,Municipal Security,NaN


In [9]:
asset_ticker['asset_type'].value_counts()

Stock                   6333
PDF Disclosed Filing     464
Municipal Security       363
Other Securities         339
Corporate Bond           218
Non-Public Stock          89
Stock Option              84
Name: asset_type, dtype: int64

### Figure out how to standardize type column

In [11]:
senators['type'].value_counts()

Purchase          4200
Sale (Full)       2064
Sale (Partial)    1735
Exchange            93
Name: type, dtype: int64

In [12]:
reps['type'].value_counts()

purchase        5114
sale_full       3283
sale_partial    1659
exchange          99
Name: type, dtype: int64

In [13]:
# Replace types in the senators data with types in the representatives data, because the rep type data is lowercase
# and in snake format
senators.loc[senators['type'] == 'Purchase', 'type'] = 'purchase'
senators.loc[senators['type'] == 'Sale (Full)', 'type'] = 'sale_full'
senators.loc[senators['type'] == 'Sale (Partial)', 'type'] = 'sale_partial'
senators.loc[senators['type'] == 'Exchange', 'type'] = 'exchange'

In [14]:
senators['type'].value_counts()

purchase        4200
sale_full       2064
sale_partial    1735
exchange          93
Name: type, dtype: int64

In [15]:
reps['type'].value_counts()

purchase        5114
sale_full       3283
sale_partial    1659
exchange          99
Name: type, dtype: int64

### Examine the date ranges for the stock transaction data for senators and representatives

In [16]:
senators['transaction_date'].sort_values()

senator
Lamar Alexander     2012-06-14
Benjamin L Cardin   2012-07-25
Thomas R Carper     2012-08-02
Robert J Portman    2012-08-15
Pat Roberts         2012-08-16
                       ...    
Mike Rounds         2021-05-01
Thomas R Carper     2021-05-07
Richard M Burr      2021-05-17
Thomas R Carper     2021-05-26
Thomas R Carper     2021-05-26
Name: transaction_date, Length: 8556, dtype: datetime64[ns]

#### We have data for Senators' transactions between June 2012 to May 2021

In [17]:
reps['transaction_date'].sort_values()

representative
Hon. Carol Devine Miller   2018-12-27
Hon. Donna Shalala         2019-01-09
Hon. Donna Shalala         2019-01-09
Hon. Donna Shalala         2019-01-09
Hon. Donna Shalala         2019-01-09
                              ...    
Hon. Mo Brooks             2021-05-14
Hon. Hakeem S. Jeffries    2021-05-17
Hon. Bob Gibbs             2021-05-25
Hon. Bob Gibbs             2021-05-25
Hon. Bob Gibbs             2021-05-25
Name: transaction_date, Length: 10155, dtype: datetime64[ns]

#### We have data for Representatives' transactions between December 2018 to May 2021